# 1 根据后缀删除dealscan中的非银行实体
规则：
- 排除以特定关键词结尾的数据
- 若Lender_Institution_Type非空，检查里面是否有"Bank"
- 若Lender_Institution_Type非空，且里面含有Bank,则不排除

In [2]:
import pandas as pd
import os

# 1. 路径设置
data_dir = 'data'
output_dir = os.path.join(data_dir, 'output')
step1_dir = os.path.join(output_dir, 'step1')

# 创建必要的目录
if not os.path.exists(step1_dir):
    os.makedirs(step1_dir)

# 2. 读取数据 (修改为读取CSV文件)
print("正在读取数据...")
df_dealscan = pd.read_csv(os.path.join(data_dir, 'dealscan_2021.csv'))
df_call = pd.read_csv(os.path.join(data_dir, 'call_2021.csv'))

# 3. 统一列名和格式
# 确保 year 和 quarter 是整数
df_dealscan['year'] = pd.to_numeric(df_dealscan['year'], errors='coerce').fillna(0).astype(int)
df_dealscan['quarter'] = pd.to_numeric(df_dealscan['quarter'], errors='coerce').fillna(0).astype(int)
df_call['year'] = pd.to_numeric(df_call['year'], errors='coerce').fillna(0).astype(int)
df_call['quarter'] = pd.to_numeric(df_call['quarter'], errors='coerce').fillna(0).astype(int)

# 4
def is_likely_target(row):
    name = row['Lender_Name']
    institution_type = row['Lender_Institution_Type']

    if pd.isna(name):
        return False

    name = str(name).strip().lower()

    # 1. 定义高风险非银行后缀（严格排除）
    # 注意：很多银行控股公司是 Corp 或 Inc，所以不要把 Corp 加入排除列表
    exclude_endings = ['fund', 'investment corp', 'llc', 'l.l.c.', 'lp', 'l.p.', 'partners', 'advisors', 'management']

    # 2. 检查是否以排除关键词结尾
    ends_with_exclude = False
    for ending in exclude_endings:
        if name.endswith(" " + ending) or name.endswith("." + ending): # 增加空格或点确保匹配完整单词
            ends_with_exclude = True
            break

    # 特殊处理：有些银行也是LLC结构（例如某些 specialized finance co），如果类型明确写了Bank，要保住
    has_bank_in_type = False
    if not pd.isna(institution_type):
        if 'bank' in str(institution_type).lower():
            has_bank_in_type = True

    # --- 逻辑核心修正 ---

    # 情况A：名字以"基金/LLC/合伙人"结尾
    if ends_with_exclude:
        # 只有当 Institution Type 明确说是 Bank 时，才保留（救回）
        if has_bank_in_type:
            return True
        else:
            return False # 否则排除

    # 情况B：名字看起来正常（不以排除词结尾）
    else:
        # 【关键修改】不要因为 Institution Type 里没有 Bank 就排除！
        # 很多银行的类型是 "Diversified Financial Services"
        # 只要名字正常，先保留，留给 Step 2 或 AI 去判断是否为美国公司
        return True

# 5. 数据筛选
print(f"筛选前 Dealscan 数量: {len(df_dealscan)}")
# 分离符合和不符合条件的数据
df_dealscan_filtered = df_dealscan[df_dealscan.apply(is_likely_target, axis=1)].copy()
df_dealscan_excluded = df_dealscan[~df_dealscan.apply(is_likely_target, axis=1)].copy()
print(f"筛选后 Dealscan 数量: {len(df_dealscan_filtered)}")

# 6. 输出筛选后的数据到指定目录 (修改为CSV格式)
filtered_output_path = os.path.join(step1_dir, 'dealscan_2021_step1.csv')
excluded_output_path = os.path.join(step1_dir, 'dealscan_2021_step1_excluded.csv')
df_dealscan_filtered.to_csv(filtered_output_path, index=False)
df_dealscan_excluded.to_csv(excluded_output_path, index=False)
print(f"✅ 筛选后的dealscan数据已保存至: {filtered_output_path}")
print(f"🗑️ 被排除的dealscan数据已保存至: {excluded_output_path}")

print("✅ 预处理完成，非银行实体已剔除。")


正在读取数据...
筛选前 Dealscan 数量: 3299
筛选后 Dealscan 数量: 3174
✅ 筛选后的dealscan数据已保存至: data\output\step1\dealscan_2021_step1.csv
🗑️ 被排除的dealscan数据已保存至: data\output\step1\dealscan_2021_step1_excluded.csv
✅ 预处理完成，非银行实体已剔除。


In [1]:
import pandas as pd
import os

# 1. 路径设置
data_dir = 'data'
output_dir = os.path.join(data_dir, 'output')
step1_dir = os.path.join(output_dir, 'step1')
check_dir = os.path.join(output_dir, 'step1', 'check')

# 创建必要的目录
if not os.path.exists(step1_dir):
    os.makedirs(step1_dir)
if not os.path.exists(check_dir):
    os.makedirs(check_dir)

# 2. 读取数据 (修改为读取CSV文件)
print("正在读取数据...")
dealscan_excluded_path = os.path.join(step1_dir, 'dealscan_2021_step1_excluded.csv')
df_dealscan_excluded = pd.read_csv(dealscan_excluded_path)
df_call = pd.read_csv(os.path.join(data_dir, 'call_2021.csv'))

# 3. 数据预处理
# 统一列名和格式
df_dealscan_excluded['Lender_Name'] = df_dealscan_excluded['Lender_Name'].astype(str).str.strip().str.lower()
df_call['name'] = df_call['name'].astype(str).str.strip().str.lower()

# 提取ID列用于精确匹配
df_dealscan_excluded['Lender_Id'] = df_dealscan_excluded['Lender_Id'].astype(str).str.strip()
df_call['rssdid'] = df_call['rssdid'].astype(str).str.strip()

print(f"被排除的dealscan数据数量: {len(df_dealscan_excluded)}")
print(f"Call报告数据数量: {len(df_call)}")

# 4. 模糊名称匹配函数
def check_fuzzy_matches():
    fuzzy_matches = []
    for _, excluded_row in df_dealscan_excluded.iterrows():
        excluded_name = excluded_row['Lender_Name']
        # 添加 regex=False 参数避免将名称解释为正则表达式
        match_found = df_call['name'].str.contains(excluded_name, case=False, na=False, regex=False).any()
        if match_found:
            fuzzy_matches.append(excluded_row.name)

    fuzzy_matches_df = df_dealscan_excluded.loc[fuzzy_matches]
    return fuzzy_matches_df

# 5. 执行模糊匹配检查
fuzzy_matches = check_fuzzy_matches()

# 6. 输出结果统计
print("\n=== 模糊名称匹配结果统计 ===")
print(f"模糊名称匹配的数量: {len(fuzzy_matches)} ({len(fuzzy_matches)/len(df_dealscan_excluded)*100:.2f}%)")

# 7. 保存匹配结果 (修改为CSV格式)
if len(fuzzy_matches) > 0:
    fuzzy_matches_output = os.path.join(check_dir, 'excluded_matched_by_fuzzy_name.csv')
    fuzzy_matches.to_csv(fuzzy_matches_output, index=False)
    print(f"\n✅ 模糊名称匹配的数据已保存至: {fuzzy_matches_output}")

# 8. 显示部分匹配示例
print("\n=== 匹配示例 ===")
if len(fuzzy_matches) > 0:
    print("模糊名称匹配示例:")
    print(fuzzy_matches[['Lender_Id', 'Lender_Name']].head(10))


正在读取数据...
被排除的dealscan数据数量: 125
Call报告数据数量: 19874

=== 模糊名称匹配结果统计 ===
模糊名称匹配的数量: 0 (0.00%)

=== 匹配示例 ===


# 2 过滤美国公司


In [3]:
import pandas as pd
import re
import os

# 配置基础参数 - 使用统一的路径格式
data_dir = 'data'
output_dir = os.path.join(data_dir, 'output')
step2_dir = os.path.join(output_dir, 'step2')

# 创建必要的目录
if not os.path.exists(step2_dir):
    os.makedirs(step2_dir)

# 使用相对路径和os.path.join统一路径格式
INPUT_FILE = os.path.join(data_dir, 'output', 'step1', 'dealscan_2021_step1.csv')
OUTPUT_US_ONLY = os.path.join(step2_dir, 'dealscan_2021_step2.csv')
OUTPUT_EXCLUDED = os.path.join(step2_dir, 'dealscan_2021_step2_excluded.csv')

# 读取原始数据 (修改为读取CSV文件)
df = pd.read_csv(INPUT_FILE)
print(f"✅ 成功读取原始文件，共 {len(df)} 条机构记录")

# 数据清洗函数
def clean_country(country_str):
    """清洗国家名称：小写+去标点+去空格"""
    if pd.isna(country_str):
        return ""
    # 转为小写→使用正则表达式去除标点符号→去除首尾空格
    cleaned = str(country_str).lower()
    # 使用re.sub()方法进行正则替换
    cleaned = re.sub(r"[^\w\s]", "", cleaned)
    cleaned = cleaned.strip()
    return cleaned

# 新增清洗后的国家字段，用于筛选
df["Country_Cleaned"] = df["Lender_Operating_Country"].apply(clean_country)

# 定义美国公司筛选规则
US_KEYWORDS_REGEX = r"^united states$|^usa$|^us$|^united states of america$|^u\.s\.a$|^u\.s$"

# 筛选美国公司：满足任一条件即可
# 1. Lender_Operating_Country为美国的各种写法
# 2. Lender_Institution_Type中含有US Bank
is_us_by_country = df["Country_Cleaned"].str.match(US_KEYWORDS_REGEX, case=False)
is_us_by_type = df["Lender_Institution_Type"].str.contains("US Bank", case=False, na=False)
is_us_company = is_us_by_country | is_us_by_type

# 分离美国公司和非美国公司
us_companies_df = df[is_us_company].copy()
excluded_df = df[~is_us_company].copy()

# 输出筛选结果统计
print("\n=== 美国公司筛选结果统计 ===")
print(f"总机构数量：{len(df):,} 家")
print(f"美国公司数量：{len(us_companies_df):,} 家")
print(f"美国公司占比：{len(us_companies_df)/len(df):.2%}")

# 显示前5家美国公司示例
print("\n=== 美国公司示例（前5条）===")
sample_columns = ["Lender_Name", "Lender_Operating_Country", "Lender_Institution_Type", "year"]
print(us_companies_df[sample_columns].head().to_string(index=False))

# 保存结果文件 (修改为CSV格式)
column_order = ["Lender_Name", "Lender_Id", "Lender_Operating_Country",
                "Lender_Institution_Type", "year", "quarter"]

# 保存仅美国公司清单
us_companies_output = us_companies_df[column_order]
us_companies_output.to_csv(OUTPUT_US_ONLY, index=False)
print(f"\n✅ 仅美国公司清单已保存：{OUTPUT_US_ONLY}")

# 保存被排除的数据文件
excluded_output = excluded_df[column_order]
excluded_output.to_csv(OUTPUT_EXCLUDED, index=False)
print(f"✅ 被排除的数据已保存至：{OUTPUT_EXCLUDED}")


✅ 成功读取原始文件，共 3174 条机构记录

=== 美国公司筛选结果统计 ===
总机构数量：3,174 家
美国公司数量：1,020 家
美国公司占比：32.14%

=== 美国公司示例（前5条）===
                          Lender_Name Lender_Operating_Country                                                                Lender_Institution_Type  year
                   Bank of America NA            United States                                     Multinational Investment Bank & Financial Services  2021
Inter American Investment Corporation            United States                                                                                    NaN  2021
                          PNC Bank NA            United States                                                                                US Bank  2021
                             Barclays           United Kingdom Asian Bank, European Bank, Multinational Investment Bank & Financial Services, US Bank  2021
              National Bank Financial            United States                                                   

# 3 AI分析
多次提问比对和检查以提高正确率

## 将dealscan_2021_step2提交给AI处理获得us_from_filtrated

## 查找us_from_filtrated在dealscan中对应的数据

In [16]:
import pandas as pd
import os

def find_missing_banks():
    """
    查找在step2结果中但不在Gemini的us_from_filtrated.txt中的银行数据
    """
    # 文件路径设置 - 使用统一的路径格式
    data_dir = 'data'
    output_dir = os.path.join(data_dir, 'output')
    step2_dir = os.path.join(output_dir, 'step2')
    step3_dir = os.path.join(output_dir, 'step3')
    check_dir = os.path.join(step3_dir, 'check')

    # 文件路径
    step2_file = os.path.join(step2_dir, 'dealscan_2021_step2.csv')
    gemini_us_file = os.path.join(step3_dir, 'us_from_filtrated.txt')

    # 创建输出目录
    for directory in [step3_dir, check_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)

    # 读取step2的美国银行数据
    print("正在读取step2筛选后的美国银行数据...")
    df_step2 = pd.read_csv(step2_file)
    print(f"✅ 成功读取step2数据，共 {len(df_step2)} 条记录")

    # 读取Gemini的us_from_filtrated.txt文件
    print("正在读取Gemini的us_from_filtrated.txt数据...")
    gemini_us_banks = set()
    if os.path.exists(gemini_us_file):
        with open(gemini_us_file, 'r', encoding='utf-8') as f:
            gemini_us_banks = {line.strip() for line in f.readlines() if line.strip()}
        print(f"✅ 成功读取Gemini US银行名单，共 {len(gemini_us_banks)} 家机构")
    else:
        print(f"❌ 未找到文件: {gemini_us_file}")
        return

    # 提取step2中的银行名称
    step2_bank_names = set(df_step2['Lender_Name'].tolist())
    print(f"📋 step2中唯一银行名称数量: {len(step2_bank_names)}")

    # 查找在step2中但不在Gemini列表中的银行
    missing_banks = step2_bank_names - gemini_us_banks

    print(f"\n=== 比对结果 ===")
    print(f"🎯 在step2中但不在Gemini US列表中的银行数量: {len(missing_banks)}")
    print(f"📊 占step2总银行数量的比例: {len(missing_banks)/len(step2_bank_names):.2%}")

    # 如果有缺失的银行，保存详细信息
    if len(missing_banks) > 0:
        # 筛选出完整的数据行
        missing_banks_df = df_step2[df_step2['Lender_Name'].isin(missing_banks)].copy()

        # 保存缺失的银行列表（仅名称）
        missing_banks_names_path = os.path.join(check_dir, "missing_banks_names.txt")
        with open(missing_banks_names_path, 'w', encoding='utf-8') as f:
            for bank_name in sorted(missing_banks):
                f.write(bank_name + '\n')

        # 保存缺失银行的完整信息
        missing_banks_full_path = os.path.join(check_dir, "missing_banks_full_info.csv")
        missing_banks_df.to_csv(missing_banks_full_path, index=False)

        print(f"\n📄 输出文件:")
        print(f"   银行名称列表已保存至: {missing_banks_names_path}")
        print(f"   完整信息已保存至: {missing_banks_full_path}")

        # 显示前10个缺失的银行示例
        print(f"\n🔍 缺失银行示例 (前10个):")
        for i, bank_name in enumerate(list(missing_banks)[:10], 1):
            print(f"   {i}. {bank_name}")
    else:
        print("✅ 所有step2中的银行都已在Gemini的US银行列表中找到")

    # 额外统计信息
    common_banks = step2_bank_names & gemini_us_banks
    print(f"\n📈 其他统计信息:")
    print(f"   两个列表中共有的银行数量: {len(common_banks)}")
    print(f"   Gemini US列表中的银行总数: {len(gemini_us_banks)}")

# 执行查找
if __name__ == "__main__":
    find_missing_banks()


正在读取step2筛选后的美国银行数据...
✅ 成功读取step2数据，共 1020 条记录
正在读取Gemini的us_from_filtrated.txt数据...
✅ 成功读取Gemini US银行名单，共 405 家机构
📋 step2中唯一银行名称数量: 1019

=== 比对结果 ===
🎯 在step2中但不在Gemini US列表中的银行数量: 621
📊 占step2总银行数量的比例: 60.94%

📄 输出文件:
   银行名称列表已保存至: data\output\step3\check\missing_banks_names.txt
   完整信息已保存至: data\output\step3\check\missing_banks_full_info.csv

🔍 缺失银行示例 (前10个):
   1. Oak Hill Capital Partners
   2. Al Ahli Bank of Kuwait KSCP
   3. US Bancorp
   4. Desjardins Florida Branch
   5. Owl Rock Capital Corp
   6. Unicredit Bank Austria AG
   7. Citigroup Global Markets Ltd
   8. JP Morgan Securities Plc
   9. Eksportfinansiering Norge
   10. CTBC Bank New York

📈 其他统计信息:
   两个列表中共有的银行数量: 398
   Gemini US列表中的银行总数: 405


## 将missing_banks_full_info提交给AI处理，获取可能被误判的银行名

大部分情况下数据较小，直接放代码里

## 补充被误删的银行（每年数据都要对应更改）

In [14]:
import os

def add_missed_banks_to_gemini():
    """
    将被误删的银行添加到Gemini的us_from_filtrated.txt文件中
    """
    # 定义需要添加的银行名称
    missed_banks = [
        "American Bank of Missouri",
        "Bangor Savings Bank",
        "Bank of Missouri",
        "First Security Bancorp / First Security Bank NA",
        "Modern Bank",
        "Salem Five Bank",
        "Summit Community Bank",
        "United Community Bank",
        "Wood Forest National Bank"
    ]

    # 文件路径设置 - 使用统一的路径格式
    data_dir = 'data'
    output_dir = os.path.join(data_dir, 'output')
    step3_dir = os.path.join(output_dir, 'step3')
    gemini_dir = os.path.join(step3_dir, 'Gemini')

    # Gemini文件路径
    gemini_file = os.path.join(gemini_dir, 'us_from_filtrated.txt')

    # 检查文件是否存在
    if not os.path.exists(gemini_file):
        print(f"❌ 未找到文件: {gemini_file}")
        return

    # 读取现有内容
    with open(gemini_file, 'r', encoding='utf-8') as f:
        existing_banks = [line.strip() for line in f.readlines() if line.strip()]

    # 转换为集合便于操作
    existing_banks_set = set(existing_banks)

    # 记录实际添加的银行
    added_banks = []

    # 添加缺失的银行（避免重复添加）
    for bank in missed_banks:
        if bank not in existing_banks_set:
            existing_banks.append(bank)
            added_banks.append(bank)
            print(f"✅ 已添加: {bank}")
        else:
            print(f"⚠️  已存在: {bank}")

    # 写回文件（保持排序）
    with open(gemini_file, 'w', encoding='utf-8') as f:
        for bank in sorted(existing_banks):
            f.write(bank + '\n')

    print(f"\n📝 总结:")
    print(f"   新增银行数量: {len(added_banks)}")
    print(f"   文件已更新: {gemini_file}")

# 执行添加操作
if __name__ == "__main__":
    add_missed_banks_to_gemini()


⚠️  已存在: American Bank of Missouri
⚠️  已存在: Bangor Savings Bank
⚠️  已存在: Bank of Missouri
⚠️  已存在: First Security Bancorp / First Security Bank NA
⚠️  已存在: Modern Bank
⚠️  已存在: Salem Five Bank
⚠️  已存在: Summit Community Bank
⚠️  已存在: United Community Bank
⚠️  已存在: Wood Forest National Bank

📝 总结:
   新增银行数量: 0
   文件已更新: data\output\step3\Gemini\us_from_filtrated.txt


## 再从当年中的call文件中补充

In [19]:
import pandas as pd
import os

def add_common_banks_to_us_list():
    """
    将call_2021.csv和dealscan_2021.csv中相同名字的机构补充到us_from_filtrated.txt中
    """
    # 文件路径设置
    data_dir = 'data'
    output_dir = os.path.join(data_dir, 'output')
    step3_dir = os.path.join(output_dir, 'step3')

    # 确保目录存在
    os.makedirs(gemini_dir, exist_ok=True)

    # 文件路径
    call_file = os.path.join(data_dir, 'call_2021.csv')
    dealscan_file = os.path.join(data_dir, 'dealscan_2021.csv')
    gemini_us_file = os.path.join(step3_dir, 'us_from_filtrated.txt')

    # 读取数据
    print("正在读取call_2021.csv...")
    df_call = pd.read_csv(call_file)
    print(f"✅ 成功读取call数据，共 {len(df_call)} 条记录")

    print("正在读取dealscan_2021.csv...")
    df_dealscan = pd.read_csv(dealscan_file)
    print(f"✅ 成功读取dealscan数据，共 {len(df_dealscan)} 条记录")

    # 提取机构名称并标准化
    # 对于call报告，使用'name'列
    call_names = set(df_call['name'].astype(str).str.strip())
    print(f"📋 call报告中唯一机构名称数量: {len(call_names)}")

    # 对于dealscan，使用'Lender_Name'列
    dealscan_names = set(df_dealscan['Lender_Name'].astype(str).str.strip())
    print(f"📋 dealscan中唯一机构名称数量: {len(dealscan_names)}")

    # 找到两个数据源中的共同机构名称
    common_names = call_names.intersection(dealscan_names)
    print(f"🔗 两个数据源中的共同机构名称数量: {len(common_names)}")

    # 读取现有的us_from_filtrated.txt文件（如果存在）
    existing_banks = set()
    if os.path.exists(gemini_us_file):
        with open(gemini_us_file, 'r', encoding='utf-8') as f:
            existing_banks = {line.strip() for line in f.readlines() if line.strip()}
        print(f"📄 已存在的US银行数量: {len(existing_banks)}")
    else:
        print("📄 us_from_filtrated.txt文件不存在，将创建新文件")

    # 找到需要新增的银行（在共同名称中但不在现有列表中）
    new_banks = common_names - existing_banks
    print(f"➕ 需要新增的银行数量: {len(new_banks)}")

    if len(new_banks) > 0:
        # 合并现有银行和新增银行
        all_banks = existing_banks.union(new_banks)

        # 写入文件（保持排序）
        with open(gemini_us_file, 'w', encoding='utf-8') as f:
            for bank in sorted(all_banks):
                f.write(bank + '\n')

        print(f"✅ 已将 {len(new_banks)} 家新银行添加到 {gemini_us_file}")
        print("新增银行示例:")
        for i, bank in enumerate(list(new_banks)[:10], 1):
            print(f"   {i}. {bank}")
    else:
        print("ℹ️  没有需要新增的银行")

    # 显示统计信息
    print("\n=== 统计摘要 ===")
    print(f"   call报告机构数量: {len(call_names)}")
    print(f"   dealscan机构数量: {len(dealscan_names)}")
    print(f"   共同机构数量: {len(common_names)}")
    print(f"   新增机构数量: {len(new_banks)}")
    print(f"   更新后总机构数量: {len(existing_banks.union(new_banks))}")

# 执行函数
if __name__ == "__main__":
    add_common_banks_to_us_list()


正在读取call_2021.csv...
✅ 成功读取call数据，共 19874 条记录
正在读取dealscan_2021.csv...
✅ 成功读取dealscan数据，共 3299 条记录
📋 call报告中唯一机构名称数量: 4437
📋 dealscan中唯一机构名称数量: 3297
🔗 两个数据源中的共同机构名称数量: 0
📄 已存在的US银行数量: 405
➕ 需要新增的银行数量: 0
ℹ️  没有需要新增的银行

=== 统计摘要 ===
   call报告机构数量: 4437
   dealscan机构数量: 3297
   共同机构数量: 0
   新增机构数量: 0
   更新后总机构数量: 405


## 检查gemini_us_from_filtrated.txt中的银行名称是否在dealscan_2021.csv中

In [8]:
import pandas as pd
import os

def filter_dealscan_by_gemini_list():
    """
    根据us_from_filtrated.txt中的银行名称筛选dealscan_2021.csv中的数据
    """
    # 文件路径设置 - 使用统一的路径格式
    data_dir = 'data'
    output_dir = os.path.join(data_dir, 'output')
    step3_dir = os.path.join(output_dir, 'step3')
    gemini_dir = os.path.join(step3_dir, 'Gemini')

    # 文件路径
    dealscan_file = os.path.join(data_dir, 'dealscan_2021.csv')
    gemini_us_file = os.path.join(gemini_dir, 'us_from_filtrated.txt')
    output_file = os.path.join(step3_dir, "dealscan_filtered_by_gemini.csv")
    not_found_file = os.path.join(step3_dir, "gemini_banks_not_in_dealscan.txt")

    # 创建输出目录（如果不存在）
    for directory in [output_dir, step3_dir, gemini_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)

    # 读取dealscan原始数据
    print("正在读取dealscan原始数据...")
    df_dealscan = pd.read_csv(dealscan_file)
    print(f"✅ 成功读取dealscan数据，共 {len(df_dealscan)} 条记录")

    # 读取Gemini的us_from_filtrated.txt文件
    print("正在读取Gemini的us_from_filtrated.txt数据...")
    gemini_us_banks = set()
    if os.path.exists(gemini_us_file):
        with open(gemini_us_file, 'r', encoding='utf-8') as f:
            gemini_us_banks = {line.strip() for line in f.readlines() if line.strip()}
        print(f"✅ 成功读取Gemini US银行名单，共 {len(gemini_us_banks)} 家机构")
    else:
        print(f"❌ 未找到文件: {gemini_us_file}")
        return

    # 筛选dealscan中包含在Gemini列表中的数据
    filtered_dealscan = df_dealscan[df_dealscan['Lender_Name'].isin(gemini_us_banks)].copy()

    # 输出统计信息
    print(f"\n=== 筛选结果 ===")
    print(f"🎯 Gemini列表中的银行在dealscan中找到记录: {len(filtered_dealscan)} 条")
    print(f"📊 占dealscan总数据比例: {len(filtered_dealscan)/len(df_dealscan):.2%}")

    # 保存筛选结果
    filtered_dealscan.to_csv(output_file, index=False)
    print(f"✅ 筛选结果已保存至: {output_file}")

    # 显示部分筛选结果示例
    print(f"\n🔍 筛选结果示例 (前5条):")
    if len(filtered_dealscan) > 0:
        sample_columns = ["Lender_Name", "Lender_Operating_Country", "Lender_Institution_Type", "year", "quarter"]
        print(filtered_dealscan[sample_columns].head().to_string(index=False))

    # 统计找不到的银行
    found_banks = set(filtered_dealscan['Lender_Name'].unique())
    not_found_banks = gemini_us_banks - found_banks

    print(f"\n❓ 在dealscan中未找到的Gemini银行数量: {len(not_found_banks)}")
    if len(not_found_banks) > 0:
        with open(not_found_file, 'w', encoding='utf-8') as f:
            for bank in sorted(not_found_banks):
                f.write(bank + '\n')
        print(f"   未找到的银行名单已保存至: {not_found_file}")

        print(f"\n📋 未找到的银行示例 (前10个):")
        for i, bank in enumerate(list(not_found_banks)[:10], 1):
            print(f"   {i}. {bank}")

# 执行筛选
if __name__ == "__main__":
    filter_dealscan_by_gemini_list()


正在读取dealscan原始数据...
✅ 成功读取dealscan数据，共 3299 条记录
正在读取Gemini的us_from_filtrated.txt数据...
✅ 成功读取Gemini US银行名单，共 405 家机构

=== 筛选结果 ===
🎯 Gemini列表中的银行在dealscan中找到记录: 398 条
📊 占dealscan总数据比例: 12.06%
✅ 筛选结果已保存至: data\output\step3\dealscan_filtered_by_gemini.csv

🔍 筛选结果示例 (前5条):
       Lender_Name Lender_Operating_Country                                                                Lender_Institution_Type  year  quarter
Bank of America NA            United States                                     Multinational Investment Bank & Financial Services  2021        1
       PNC Bank NA            United States                                                                                US Bank  2021        1
    Northern Trust            United States                            Multinational Investment Bank & Financial Services, US Bank  2021        1
         Citigroup            United States Asian Bank, European Bank, Multinational Investment Bank & Financial Services, US Bank  2021        1


# 4 生成变体名(这个需要每次根据年份数据扩充打表）

In [11]:
import pandas as pd
import re
import os

# 文件路径设置 - 使用统一的路径格式
data_dir = 'data'
output_dir = os.path.join(data_dir, 'output')
step4_dir = os.path.join(output_dir, 'step4')
gemini_dir = os.path.join(step4_dir, 'Gemini')

# 创建必要的目录
for directory in [output_dir, step4_dir, gemini_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# 1. 读取原始数据
# 假设你的文件名为 us_from_filtrated.txt，每行一个名字
input_file = os.path.join(gemini_dir, "us_from_filtrated.txt")
output_file = os.path.join(gemini_dir, "us_bank_mapping_table.csv")

try:
    with open(input_file, "r", encoding="utf-8") as f:
        raw_names = [line.strip() for line in f.readlines() if line.strip()]
except FileNotFoundError:
    print(f"Error: 找不到文件 {input_file}，请确保文件在当前目录下。")
    raw_names = []

# 2. 定义核心知识库 (Knowledge Base)
# 格式: '关键词/正则': {'Standard': '标准名', 'Parent': '母公司', 'Type': '备注/并购'}
# 注意：匹配顺序很重要，特殊的放在前面
mapping_rules = {
    # --- 特殊集团与多品牌 ---
    r"Zions|Amegy|Vectra|California Bank & Trust|National Bank of Arizona|Nevada State Bank": {
        "Standard": "Zions Bancorporation, N.A.",
        "Parent": "Zions Bancorporation",
        "Notes": "Amegy, Vectra, CB&T 均为 Zions 旗下品牌"
    },
    r"BOK|Bank of Oklahoma|Bank of Texas|Bank of Albuquerque": {
        "Standard": "BOKF, National Association",
        "Parent": "BOK Financial Corp",
        "Notes": "运营品牌包括 Bank of Oklahoma, Bank of Texas 等"
    },
    r"Wintrust|Lake Forest|Hinsdale|Wheaton|Barrington": {
        "Standard": "Wintrust Bank, N.A.",
        "Parent": "Wintrust Financial Corp",
        "Notes": "Wintrust 拥有众多社区银行品牌"
    },

    # --- 大型银行与合并 ---
    r"BBVA|Compass": {
        "Standard": "BBVA USA",
        "Parent": "BBVA (Spain)",
        "Notes": "2021年被 PNC 收购，此前为 BBVA USA"
    },
    r"SunTrust": {
        "Standard": "Truist Bank",
        "Parent": "Truist Financial",
        "Notes": "SunTrust 与 BB&T 合并为 Truist"
    },
    r"IberiaBank": {
        "Standard": "First Horizon Bank",
        "Parent": "First Horizon Corp",
        "Notes": "已被 First Horizon 收购"
    },
    r"TCF": {
        "Standard": "TCF National Bank",
        "Parent": "Huntington Bancshares",
        "Notes": "2021年并入 Huntington"
    },
    r"Sterling National": {
        "Standard": "Sterling National Bank",
        "Parent": "Webster Financial",
        "Notes": "2022年并入 Webster Bank"
    },
    r"Boston Private": {
        "Standard": "Boston Private Bank & Trust Company",
        "Parent": "SVB Financial / First Citizens",
        "Notes": "2021年被 SVB 收购"
    },
    r"People's United": {
        "Standard": "People's United Bank, N.A.",
        "Parent": "M&T Bank Corp",
        "Notes": "2022年并入 M&T"
    },
    r"Bank Leumi": {
        "Standard": "Bank Leumi USA",
        "Parent": "Valley National Bancorp",
        "Notes": "2022年并入 Valley National"
    },

    # --- 拼写错误修正 ---
    r"Renesant": {
        "Standard": "Renasant Bank",
        "Parent": "Renasant Corp",
        "Notes": "纠正拼写 (Renesant -> Renasant)"
    },
    r"Synovous": {
        "Standard": "Synovus Bank",
        "Parent": "Synovus Financial Corp",
        "Notes": "纠正拼写 (Synovous -> Synovus)"
    },
    r"First Womens": {
        "Standard": "First Women's Bank",
        "Parent": "",
        "Notes": "纠正拼写 (加撇号)"
    },

    # --- 常见银行实体标准化 ---
    r"Bank of America": {"Standard": "Bank of America, N.A.", "Parent": "Bank of America Corp"},
    r"JP Morgan|JPMorgan|Chase": {"Standard": "JPMorgan Chase Bank, N.A.", "Parent": "JPMorgan Chase & Co"},
    r"Wells Fargo": {"Standard": "Wells Fargo Bank, N.A.", "Parent": "Wells Fargo & Co"},
    r"Citi": {"Standard": "Citibank, N.A.", "Parent": "Citigroup Inc"},
    r"PNC": {"Standard": "PNC Bank, National Association", "Parent": "PNC Financial Services"},
    r"US Bank": {"Standard": "U.S. Bank National Association", "Parent": "U.S. Bancorp"},
    r"Truist": {"Standard": "Truist Bank", "Parent": "Truist Financial Corp"},
    r"TD Bank": {"Standard": "TD Bank, N.A.", "Parent": "TD Group US"},
    r"Capital One": {"Standard": "Capital One, N.A.", "Parent": "Capital One Financial"},
    r"Fifth Third": {"Standard": "Fifth Third Bank, N.A.", "Parent": "Fifth Third Bancorp"},
    r"KeyBank": {"Standard": "KeyBank National Association", "Parent": "KeyCorp"},
    r"Citizens": {"Standard": "Citizens Bank, N.A.", "Parent": "Citizens Financial Group"},
    r"M&T": {"Standard": "Manufacturers and Traders Trust Company", "Parent": "M&T Bank Corp"},
    r"Huntington": {"Standard": "The Huntington National Bank", "Parent": "Huntington Bancshares"},
    r"Regions": {"Standard": "Regions Bank", "Parent": "Regions Financial"},
    r"Ally": {"Standard": "Ally Bank", "Parent": "Ally Financial"},
    r"Goldman Sachs": {"Standard": "Goldman Sachs Bank USA", "Parent": "Goldman Sachs Group"},
    r"Morgan Stanley": {"Standard": "Morgan Stanley Bank, N.A.", "Parent": "Morgan Stanley"},
    r"State Street": {"Standard": "State Street Bank and Trust Company", "Parent": "State Street Corp"},
    r"Northern Trust": {"Standard": "The Northern Trust Company", "Parent": "Northern Trust Corp"},
    r"NY Mellon|New York Mellon": {"Standard": "The Bank of New York Mellon", "Parent": "BNY Mellon"},
    r"Silicon Valley|SVB": {"Standard": "Silicon Valley Bank", "Parent": "SVB Financial Group", "Notes": "2021实体"},
    r"Signature": {"Standard": "Signature Bank", "Parent": "", "Notes": "2021实体"},
    r"First Republic": {"Standard": "First Republic Bank", "Parent": "", "Notes": "2021实体"},
    r"MUFG|Union Bank": {"Standard": "MUFG Union Bank, N.A.", "Parent": "Mitsubishi UFJ", "Notes": "现并入 U.S. Bank"},
    r"BMO": {"Standard": "BMO Harris Bank N.A.", "Parent": "BMO Financial Group"},
    r"HSBC": {"Standard": "HSBC Bank USA, N.A.", "Parent": "HSBC Holdings"},
    r"Santander": {"Standard": "Santander Bank, N.A.", "Parent": "Santander Group"},
    r"Deutsche Bank": {"Standard": "Deutsche Bank Trust Company Americas", "Parent": "Deutsche Bank AG"},

    # --- 区域银行标准化 ---
    r"Comerica": {"Standard": "Comerica Bank", "Parent": "Comerica Inc"},
    r"First Horizon": {"Standard": "First Horizon Bank", "Parent": "First Horizon Corp"},
    r"Synovus": {"Standard": "Synovus Bank", "Parent": "Synovus Financial Corp"},
    r"Western Alliance": {"Standard": "Western Alliance Bank", "Parent": "Western Alliance Bancorp"},
    r"East West": {"Standard": "East West Bank", "Parent": "East West Bancorp"},
    r"First Citizens": {"Standard": "First-Citizens Bank & Trust Company", "Parent": "First Citizens BancShares"},
    r"Cullen|Frost": {"Standard": "Frost Bank", "Parent": "Cullen/Frost Bankers"},
    r"South State|SouthState": {"Standard": "SouthState Bank, N.A.", "Parent": "SouthState Corp"},
    r"FNB|First National Bank of Pennsylvania": {"Standard": "First National Bank of Pennsylvania", "Parent": "F.N.B. Corp"},
    r"Webster": {"Standard": "Webster Bank, N.A.", "Parent": "Webster Financial"},
    r"Old National": {"Standard": "Old National Bank", "Parent": "Old National Bancorp"},
    r"Prosperity": {"Standard": "Prosperity Bank", "Parent": "Prosperity Bancshares"},
    r"Glacier": {"Standard": "Glacier Bank", "Parent": "Glacier Bancorp"},
    r"Pinnacle": {"Standard": "Pinnacle Bank", "Parent": "Pinnacle Financial Partners"},
    r"Umb": {"Standard": "UMB Bank, n.a.", "Parent": "UMB Financial"},
    r"Commerce Bank": {"Standard": "Commerce Bank", "Parent": "Commerce Bancshares"},
    r"Associated": {"Standard": "Associated Bank, N.A.", "Parent": "Associated Banc-Corp"},
    r"Hancock|Whitney": {"Standard": "Hancock Whitney Bank", "Parent": "Hancock Whitney Corp"},
    r"Wintrust": {"Standard": "Wintrust Bank, N.A.", "Parent": "Wintrust Financial"},
    r"Valley National": {"Standard": "Valley National Bank", "Parent": "Valley National Bancorp"},
    r"Texas Capital": {"Standard": "Texas Capital Bank", "Parent": "Texas Capital Bancshares"},
    r"Fulton": {"Standard": "Fulton Bank, N.A.", "Parent": "Fulton Financial"},
    r"Simmons": {"Standard": "Simmons Bank", "Parent": "Simmons First National"},
    r"Cadence": {"Standard": "Cadence Bank", "Parent": "Cadence Bank"},
    r"United Bank": {"Standard": "United Bank", "Parent": "United Bankshares"},
    r"First Hawaiian": {"Standard": "First Hawaiian Bank", "Parent": "First Hawaiian Inc"},
    r"Cathay": {"Standard": "Cathay Bank", "Parent": "Cathay General Bancorp"},
    r"PacWest|Pacific Western": {"Standard": "Pacific Western Bank", "Parent": "PacWest Bancorp", "Notes": "现 Banc of California"},
    r"Washington Federal|WaFd": {"Standard": "Washington Federal Bank", "Parent": "Washington Federal Inc"},
    r"Bank of Hawaii": {"Standard": "Bank of Hawaii", "Parent": "Bank of Hawaii Corp"},
    r"Home BancShares|Centennial": {"Standard": "Centennial Bank", "Parent": "Home BancShares"},
    r"Atlantic Union": {"Standard": "Atlantic Union Bank", "Parent": "Atlantic Union Bankshares"},
    r"Independent Bank": {"Standard": "Independent Bank", "Parent": "Independent Bank Group"},
    r"TowneBank": {"Standard": "TowneBank", "Parent": ""},
    r"Eastern": {"Standard": "Eastern Bank", "Parent": "Eastern Bankshares"},
    r"Ameris": {"Standard": "Ameris Bank", "Parent": "Ameris Bancorp"},
    r"First Financial": {"Standard": "First Financial Bank, N.A.", "Parent": "First Financial Bancorp"},
    r"First Merchants": {"Standard": "First Merchants Bank", "Parent": "First Merchants Corp"},
    r"MidFirst": {"Standard": "MidFirst Bank", "Parent": "Midland Financial"},
    r"Arvest": {"Standard": "Arvest Bank", "Parent": ""},
    r"Flagstar": {"Standard": "Flagstar Bank, N.A.", "Parent": "NY Community Bancorp"},
    r"First Interstate": {"Standard": "First Interstate Bank", "Parent": "First Interstate BancSystem"},
    r"Renasant": {"Standard": "Renasant Bank", "Parent": "Renasant Corp"},
    r"NBT": {"Standard": "NBT Bank, N.A.", "Parent": "NBT Bancorp"},
    r"Customers": {"Standard": "Customers Bank", "Parent": "Customers Bancorp"},
    r"BancFirst": {"Standard": "BancFirst", "Parent": "BancFirst Corp"},
    r"Trustmark": {"Standard": "Trustmark National Bank", "Parent": "Trustmark Corp"},
    r"WesBanco": {"Standard": "WesBanco Bank, Inc.", "Parent": "WesBanco Inc"},
    r"Provident": {"Standard": "Provident Bank", "Parent": "Provident Financial"},
    r"Heartland": {"Standard": "Heartland Bank", "Parent": "Heartland Financial USA"},
    r"Sandy Spring": {"Standard": "Sandy Spring Bank", "Parent": "Sandy Spring Bancorp"},
    r"First National Bank of Omaha": {"Standard": "First National Bank of Omaha", "Parent": "First National of Nebraska"},
    r"Axos": {"Standard": "Axos Bank", "Parent": "Axos Financial"},
    r"Tri Counties": {"Standard": "Tri counties Bank", "Parent": "TriCo Bancshares"},
    r"EagleBank|Eagle Bancorp": {"Standard": "EagleBank", "Parent": "Eagle Bancorp"},
    r"Brookline": {"Standard": "Brookline Bank", "Parent": "Brookline Bancorp"},
    r"ServisFirst": {"Standard": "ServisFirst Bank", "Parent": "ServisFirst Bancshares"},
    r"First Commonwealth": {"Standard": "First Commonwealth Bank", "Parent": "First Commonwealth Fin"},
    r"Park National": {"Standard": "The Park National Bank", "Parent": "Park National Corp"},
    r"City National": {"Standard": "City National Bank", "Parent": "Royal Bank of Canada"},
    r"CIBC": {"Standard": "CIBC Bank USA", "Parent": "CIBC"},
    r"Israel Discount|IDB": {"Standard": "Israel Discount Bank of New York", "Parent": "Discount Bancorp"},
    r"Apple Bank": {"Standard": "Apple Bank for Savings", "Parent": ""},
    r"Amalgamated": {"Standard": "Amalgamated Bank", "Parent": ""},
    r"TIAA": {"Standard": "TIAA, FSB", "Parent": "TIAA", "Notes": "2021年名称"},
    r"Charles Schwab": {"Standard": "Charles Schwab Bank, SSB", "Parent": "Charles Schwab Corp"},
    r"American Savings": {"Standard": "American Savings Bank, F.S.B.", "Parent": "Hawaiian Electric"},
    r"Banc of California": {"Standard": "Banc of California, N.A.", "Parent": "Banc of California Inc"},
    r"First Foundation": {"Standard": "First Foundation Bank", "Parent": "First Foundation Inc"},
    r"Dime": {"Standard": "Dime Community Bank", "Parent": "Dime Community Bancshares"},
    r"Hope": {"Standard": "Bank of Hope", "Parent": "Hope Bancorp"},
    r"Hanmi": {"Standard": "Hanmi Bank", "Parent": "Hanmi Financial"},
    r"Pacific Premier": {"Standard": "Pacific Premier Bank", "Parent": "Pacific Premier Bancorp"},
    r"CVB|Citizens Business": {"Standard": "Citizens Business Bank", "Parent": "CVB Financial"},
    r"Mechanics": {"Standard": "Mechanics Bank", "Parent": ""},
    r"Banner": {"Standard": "Banner Bank", "Parent": "Banner Corp"},
    r"Columbia": {"Standard": "Columbia State Bank", "Parent": "Columbia Banking System"},
    r"Heritage": {"Standard": "Heritage Bank", "Parent": "Heritage Financial"},
    r"Umpqua": {"Standard": "Umpqua Bank", "Parent": "Umpqua Holdings"},
    r"Washington Trust": {"Standard": "The Washington Trust Company", "Parent": "Washington Trust Bancorp"},
    r"First Busey|Busey": {"Standard": "Busey Bank", "Parent": "First Busey Corp"},
    r"Great Southern": {"Standard": "Great Southern Bank", "Parent": "Great Southern Bancorp"},
    r"Enterprise": {"Standard": "Enterprise Bank & Trust", "Parent": "Enterprise Financial"},
    r"QCR": {"Standard": "Quad City Bank & Trust", "Parent": "QCR Holdings"},
    r"Midland": {"Standard": "Midland States Bank", "Parent": "Midland States Bancorp"},
    r"First Mid": {"Standard": "First Mid Bank & Trust", "Parent": "First Mid Bancshares"},
    r"Horizon": {"Standard": "Horizon Bank", "Parent": "Horizon Bancorp"},
    r"CrossFirst": {"Standard": "CrossFirst Bank", "Parent": "CrossFirst Bankshares"},
    r"Equity": {"Standard": "Equity Bank", "Parent": "Equity Bancshares"},
    r"Origin": {"Standard": "Origin Bank", "Parent": "Origin Bancorp"},
    r"Veritex": {"Standard": "Veritex Community Bank", "Parent": "Veritex Holdings"},
    r"Triumph": {"Standard": "Triumph Bank", "Parent": "Triumph Bancorp"},
    r"Allegiance": {"Standard": "Allegiance Bank", "Parent": "Allegiance Bancshares"},
    r"Stellantis": {"Standard": "Stellantis Financial Services", "Parent": ""},
    r"Toyota": {"Standard": "Toyota Financial Savings Bank", "Parent": "Toyota"},
    r"BMW": {"Standard": "BMW Bank of North America", "Parent": "BMW"},
    r"UBS": {"Standard": "UBS Bank USA", "Parent": "UBS Group"},
    r"American Express": {"Standard": "American Express National Bank", "Parent": "American Express"},
    r"Discover": {"Standard": "Discover Bank", "Parent": "Discover Financial"},
    r"Sallie Mae": {"Standard": "Sallie Mae Bank", "Parent": "SLM Corp"},
    r"Synchrony": {"Standard": "Synchrony Bank", "Parent": "Synchrony Financial"},
    r"Barclays": {"Standard": "Barclays Bank Delaware", "Parent": "Barclays PLC"},
    r"Live Oak": {"Standard": "Live Oak Bank", "Parent": "Live Oak Bancshares"},
    r"Silvergate": {"Standard": "Silvergate Bank", "Parent": "Silvergate Capital"},
    r"Metropolitan": {"Standard": "Metropolitan Commercial Bank", "Parent": "Metropolitan Bank Holding"},
    r"ConnectOne": {"Standard": "ConnectOne Bank", "Parent": "ConnectOne Bancorp"},
    r"Peapack": {"Standard": "Peapack-Gladstone Bank", "Parent": "Peapack-Gladstone Fin"},
    r"Lakeland": {"Standard": "Lakeland Bank", "Parent": "Lakeland Bancorp"},
    r"OceanFirst": {"Standard": "OceanFirst Bank", "Parent": "OceanFirst Financial"},
    r"Kearny": {"Standard": "Kearny Bank", "Parent": "Kearny Financial"},
    r"Nicolet": {"Standard": "Nicolet National Bank", "Parent": "Nicolet Bankshares"},
    r"1st Source|First Source": {"Standard": "1st Source Bank", "Parent": "1st Source Corp"},
    r"Cambridge": {"Standard": "Cambridge Savings Bank", "Parent": "Cambridge Bancorp"},
    r"HarborOne": {"Standard": "HarborOne Bank", "Parent": "HarborOne Bancorp"},
    r"Berkshire": {"Standard": "Berkshire Bank", "Parent": "Berkshire Hills Bancorp"},
    r"Rockland": {"Standard": "Rockland Trust Company", "Parent": "Independent Bank Corp"},
    r"Seacoast": {"Standard": "Seacoast National Bank", "Parent": "Seacoast Banking Corp"},
    r"United Community": {"Standard": "United Community Bank", "Parent": "United Community Banks"},
    r"Community Bank": {"Standard": "Community Bank, N.A.", "Parent": "Community Bank System"},
    r"Tompkins": {"Standard": "Tompkins Community Bank", "Parent": "Tompkins Financial"},
    r"Arrow": {"Standard": "Glens Falls National Bank", "Parent": "Arrow Financial"},
    r"TrustCo": {"Standard": "Trustco Bank", "Parent": "TrustCo Bank Corp NY"},
    r"Northwest": {"Standard": "Northwest Bank", "Parent": "Northwest Bancshares"},
    r"S&T": {"Standard": "S&T Bank", "Parent": "S&T Bancorp"},
    r"City Holding": {"Standard": "City National Bank of West Virginia", "Parent": "City Holding Co"},
    r"Stock Yards": {"Standard": "Stock Yards Bank & Trust", "Parent": "Stock Yards Bancorp"},
    r"Republic": {"Standard": "Republic Bank & Trust Company", "Parent": "Republic Bancorp"},
    r"Civista": {"Standard": "Civista Bank", "Parent": "Civista Bancshares"},
    r"Farmers & Merchants": {"Standard": "Farmers & Merchants Bank of Long Beach", "Parent": "F&M Bank Corp"},
    r"Westamerica": {"Standard": "Westamerica Bank", "Parent": "Westamerica Bancorporation"},
    r"TriCo": {"Standard": "Tri Counties Bank", "Parent": "TriCo Bancshares"},
    r"Hana": {"Standard": "KEB Hana Bank USA", "Parent": "Hana Financial"},
    r"Woori": {"Standard": "Woori America Bank", "Parent": "Woori Financial"},
    r"Shinhan": {"Standard": "Shinhan Bank America", "Parent": "Shinhan Financial"},
    r"Industrial and Commercial": {"Standard": "Industrial and Commercial Bank of China (USA)", "Parent": "ICBC"},
    r"Bank of China": {"Standard": "Bank of China", "Parent": "Bank of China", "Notes": "Branch Status"},
    r"Mizuho": {"Standard": "Mizuho Bank (USA)", "Parent": "Mizuho FG"},
    r"Sumitomo|SMBC": {"Standard": "Manufacturers Bank", "Parent": "SMBC", "Notes": "SMBC美国子公司"},
    r"Modern Bank": {"Standard": "Modern Bank, N.A.", "Parent": ""},
    r"Woodforest": {"Standard": "Woodforest National Bank", "Parent": ""},
    r"Salem Five": {"Standard": "Salem Five Cents Savings Bank", "Parent": ""},
    r"Academy": {"Standard": "Academy Bank, N.A.", "Parent": "Dickinson Financial"},
    r"Cross River": {"Standard": "Cross River Bank", "Parent": "CRB Group"},
    r"Green Dot": {"Standard": "Green Dot Bank", "Parent": "Green Dot Corp"},
    r"Celtic": {"Standard": "Celtic Bank", "Parent": ""},
    r"Forbright": {"Standard": "Forbright Bank", "Parent": "Forbright Inc"},
    r"Congressional": {"Standard": "Forbright Bank", "Parent": "Forbright Inc", "Notes": "已更名"},
}

# 3. 处理函数
processed_data = []

for raw_name in raw_names:
    matched = False

    # 3.1 尝试匹配知识库
    for pattern, info in mapping_rules.items():
        if re.search(pattern, raw_name, re.IGNORECASE):
            processed_data.append({
                "Original Name": raw_name,
                "FDIC/RSSD Standard Name": info['Standard'],
                "Holding Company": info['Parent'],
                "Variations/Aliases": pattern.replace(r"|", ", "),
                "Notes": info.get('Notes', '')
            })
            matched = True
            break

    # 3.2 如果没有匹配到知识库，进行通用清洗
    if not matched:
        clean_name = raw_name
        # 通用后缀处理
        if " NA" in clean_name and ", N.A." not in clean_name:
            clean_name = clean_name.replace(" NA", ", N.A.")
        elif " Bank" in clean_name and "NA" not in clean_name and "Company" not in clean_name:
            # 这是一个猜测，很多时候加上 N.A. 是安全的，或者保持原样
            pass

        processed_data.append({
            "Original Name": raw_name,
            "FDIC/RSSD Standard Name": clean_name, # 默认保留原名或简单清洗
            "Holding Company": "",
            "Variations/Aliases": "",
            "Notes": "Auto-cleaned, check manually"
        })

# 4. 导出结果
df_result = pd.DataFrame(processed_data)
df_result.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"处理完成！已生成映射表：{output_file}")
print(f"共处理 {len(raw_names)} 条数据。")


Error: 找不到文件 data\output\step4\Gemini\us_from_filtrated.txt，请确保文件在当前目录下。
处理完成！已生成映射表：data\output\step4\Gemini\us_bank_mapping_table.csv
共处理 0 条数据。
